<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/CS_Gov_Class1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

#
import pandas as pd
import requests

# URL for the WGI dataset on the World Bank Open Data website
url = 'http://databank.worldbank.org/data/download/ggda/c6/c68a1c14-62fe-40d6-988c-03d83c35105a/8b8d97'

# Download the CSV file and save it to the local machine
response = requests.get(url)
with open('world_gov_indicators_raw.csv', 'wb') as f:
    f.write(response.content)

# Load the CSV file into a pandas dataframe
df = pd.read_csv('world_gov_indicators_raw.csv', header=2)

# Delete any irrelevant columns
df.drop(['Country Code', 'Indicator Code'], axis=1, inplace=True)

# Rename columns to lowercase and English headers
df.rename(columns={'Country Name': 'country', 'Indicator Name': 'indicator'}, inplace=True)

# Save the cleaned dataframe to a new CSV file
df.to_csv('world_gov_indicators_cleaned.csv', index=False)

# Load data
df = pd.read_csv('government_data.csv')

# Drop unnecessary columns
df.drop(['Country Code', 'Indicator Name', 'Unnamed: 65'], axis=1, inplace=True)

# Pivot data to wide format
df = df.pivot(index='Country Name', columns='Indicator Code', values='2018')

# Remove rows with missing values
df.dropna(inplace=True)

# Standardize data using MinMaxScaler
scaler = MinMaxScaler()
df_std = pd.DataFrame(scaler.fit_transform(df), index=df.index, columns=df.columns)

# Determine optimal number of clusters using elbow method
distortions = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=0).fit(df_std)
    distortions.append(kmeans.inertia_)

# Visualize elbow plot
import matplotlib.pyplot as plt
plt.plot(range(1, 11), distortions)
plt.title('Elbow Plot')
plt.xlabel('Number of Clusters')
plt.ylabel('Distortion')
plt.show()

# Perform clustering with optimal number of clusters
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_std)
clusters = kmeans.labels_

# Output cluster labels
result = pd.DataFrame({'Country Name': df.index, 'Cluster': clusters})
print(result)
